In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from sklearn.neural_network import MLPClassifier
from keras.models import Sequential
from keras.layers import Dense, Flatten
from sklearn.metrics import precision_recall_fscore_support,confusion_matrix
import numpy as np

import keras_metrics
import keras


In [ ]:
train_datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

In [ ]:
valid_datagen = ImageDataGenerator(rescale=1./255)

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = valid_datagen.flow_from_directory(
        'dataset/test',
        target_size=(150, 150),
        color_mode= "grayscale",
        batch_size=32,
        class_mode='categorical')
train_generator = train_datagen.flow_from_directory(
        'dataset/train',  # this is the target directory
        target_size=(150, 150),  # all images will be resized to 150x150
        color_mode= "grayscale",
        batch_size=32,
        class_mode='categorical')  # since we use binary_crossentropy loss, we need binary labels

# this is a similar generator, for validation data
valid_generator = valid_datagen.flow_from_directory(
        'dataset/validate',
        target_size=(150, 150),
        color_mode= "grayscale",
        batch_size=32,
        class_mode='categorical')


In [ ]:
# Function to create model, required for KerasClassifier


model = Sequential()
model.add(Flatten(input_shape=(150,150)))
model.add(Dense(150, input_dim=22500, activation='relu'))
model.add(Dense(150, activation='relu'))
model.add(Dense(150, activation='relu'))
model.add(Dense(150, activation='relu'))
model.add(Dense(150, activation='relu'))
model.add(Dense(10, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=["accuracy",keras.metrics.Precision(), keras.metrics.Recall()])
model.summary()

In [ ]:
model.fit_generator(
        train_generator,
        steps_per_epoch=800 // 32,
        epochs=500,
        validation_data=valid_generator,
        validation_steps=200 // 32)
model.save_weights('first_try.h5') 

In [ ]:
filenames = test_generator.filenames
nb_samples = len(filenames)
pred = model.predict_generator(test_generator)#,steps = nb_samples)

In [ ]:
val_preds = np.argmax(pred, axis=-1)
val_trues = test_generator.classes
cm = confusion_matrix(val_trues, val_preds)
cm
print(val_trues)
print(val_preds)

In [ ]:
labels = test_generator.class_indices.keys()
labels

In [ ]:
precisions, recall, f1_score, _ = precision_recall_fscore_support(val_trues, val_preds)

In [ ]:
recall

In [ ]:
precisions

In [ ]:
import numpy as np
import cv2
  
     
# Read the query image as query_img
# and train image This query image
# is what you need to find in train image
# Save it in the same directory
# with the name image.jpg 
train_img = cv2.imread('dataset/test/00004/00719_00002.ppm')
query_img = cv2.imread('dataset/train/00004/01017_00001.ppm')

# print(trainDescriptors.shape)

    
# Convert it to grayscale
query_img_bw = cv2.cvtColor(query_img,cv2.COLOR_BGR2GRAY)
train_img_bw = cv2.cvtColor(train_img, cv2.COLOR_BGR2GRAY)
  
# Initialize the ORB detector algorithm
orb = cv2.ORB_create()
  
# Now detect the keypoints and compute
# the descriptors for the query image
# and train image
queryKeypoints, queryDescriptors = orb.detectAndCompute(query_img_bw,None)
trainKeypoints, trainDescriptors = orb.detectAndCompute(train_img_bw,None)
print("query descriptors" , queryDescriptors.shape)
print("train Descriptor shape  = ", trainDescriptors.shape)
 
# Initialize the Matcher for matching
# the keypoints and then match the
# keypoints
matcher = cv2.BFMatcher()
matches = matcher.match(queryDescriptors,trainDescriptors)
  
# draw the matches to the final image
# containing both the images the drawMatches()
# function takes both images and keypoints
# and outputs the matched query image with
# its train image
final_img = cv2.drawMatches(query_img, queryKeypoints,
train_img, trainKeypoints, matches[:20],None)
  
final_img = cv2.resize(final_img, (1000,650))
 
# Show the final image
cv2.imshow("Matches", final_img)
cv2.waitKey(3000)

In [44]:
import os
import numpy as np
import cv2
from sklearn import svm 
import numpy as np 
from skimage.feature import hog 
trainKeyposints = []
trainDescriptors = []
labels = []
orb = cv2.ORB_create()

hog_train_data = [] 
# hog = cv2.HOGDescriptor()


for folder in os.listdir("dataset/train/"):
    print(folder)
    
    for file in os.listdir("dataset/train/" + folder + "/"):
#         print(file)
        name = 'dataset/train/'+ folder +"/" + file
#         print(name)
        train_img = cv2.imread('dataset/train/'+ folder +"/" + file)
        if train_img is None:
            print("no image")
#         cv2.imshow("imgt",train_img)
#         cv2.waitKey(0)
        dim = (32, 32)
  
        # resize image
        img = cv2.resize(train_img, dim, interpolation = cv2.INTER_AREA)

#         h = hog.compute(img)
#         winStride = (8,8)
#         padding = (8,8)
#         locations = ((10,20),)
#         hist = hog.compute(img,winStride,padding,locations)

 
#         convert BGR image to gray-scale 
#         img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) 
#         tune these hyper-parameters according to your needs 
#         here fd is the new feature descriptor for your image 
        fd, hog_image = hog(img, orientations=9, pixels_per_cell=(8, 8), 
                    cells_per_block=(2, 2), visualize=True, multichannel=True)
#         for visualisation of the hog features 
#         plt.imshow(hog_img) 
        hog_train_data.append(fd) 
        
#         train_img_bw = cv2.cvtColor(train_img, cv2.COLOR_BGR2GRAY)

        
#         Keypoints, Descriptors = orb.detectAndCompute(train_img_bw,None)
#         if Descriptors is not None:
#             print(Descriptors.shape)
#             trainDescriptors.append(Descriptors)#.flatten())
#             trainKeyposints.append(Keypoints)
        labels.append(int(folder))
# #             print(len(Descriptors.flatten()))
#         else: 
#             print("NONE Descriptor")
#             pass


00004
00002
00006
00001
00007
00008
00003
00005
00010
00009


In [45]:
#convert it to np array 
hog_train_data = np.array(hog_train_data) 
#one-vs-all SVM linear classifier 
lin_clf = svm.LinearSVC() 
#train the classifier using new hog features and training labels 
lin_clf.fit(hog_train_data, labels) 
#likewise extract features for your testing data and test the classifier using the new features 

LinearSVC()

In [48]:
test_img = cv2.imread('dataset/test/00004/00719_00002.ppm')
if train_img is None:
    print("no image")
#         cv2.imshow("imgt",train_img)
#         cv2.waitKey(0)
dim = (32, 32)

# resize image
img = cv2.resize(test_img, dim, interpolation = cv2.INTER_AREA)
fd, hog_image = hog(img, orientations=9, pixels_per_cell=(8, 8), 
                    cells_per_block=(2, 2), visualize=True, multichannel=True)




pred = lin_clf.predict([fd])

In [49]:
pred

array([4])

In [ ]:
# from sklearn.cluster import KMeans

# kmeans = KMeans(n_clusters = 200, n_init=10, init='random')
# gg=[item for sublist in trainDescriptors for item in sublist]
# kmeans.fit(gg)#vocabulary

In [ ]:
# from sklearn import svm
# clf = svm.SVC(decision_function_shape='ovo')
# clf.fit(trainDescriptors, labels)

In [ ]:

# query_img = cv2.imread('dataset/test/00004/00719_00002.ppm')
  


    
# # Convert it to grayscale
# query_img_bw = cv2.cvtColor(query_img,cv2.COLOR_BGR2GRAY)

  
# # Initialize the ORB detector algorithm
# orb = cv2.ORB_create()
  
# # Now detect the keypoints and compute
# # the descriptors for the query image
# # and train image
# queryKeypoints, queryDescriptors = orb.detectAndCompute(query_img_bw,None)

# print(len(queryDescriptors))


# clf.predict([queryDescriptors.flatten()])

In [ ]:
# import numpy as np
# import cv2 as cv
# img = cv.imread('dataset/train/00004/01017_00001.ppm')
# gray= cv.cvtColor(img,cv.COLOR_BGR2GRAY)
# sift = cv.SIFT_create()
# kp = sift.detect(gray,None)
# img=cv.drawKeypoints(gray,kp,img)
# cv.imwrite('sift_keypoints.jpg',img)

In [ ]:
for i in range(0,len(trainDescriptors)):
    print(len(trainDescriptors[i]))

In [ ]:
#Assume shape of train_data: (no. of samples) x (32*32*3) where 32x32x3 is the dimension of a BGR image 
import cv2 
from sklearn import svm 
import numpy as np 
from skimage.feature import hog 
import matplotlib.pyplot as plt 
 
hog_train_data = [] 
for i in range(len(train_data)): 
    img = train_data[i].reshape(32, 32, 3) 
	#convert BGR image to gray-scale 
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) 
	#tune these hyper-parameters according to your needs 
	#here fd is the new feature descriptor for your image 
    fd, hog_img = hog(img, orientations=10, pixels_per_cell=(12, 12), cells_per_block=(8, 8), feature_vector=True, visualise=True, block_norm='L2-Hys') 
	#for visualisation of the hog features 
	plt.imshow(hog_img) 
    hog_train_data.append(fd) 
#convert it to np array 
hog_train_data = np.array(hog_train_data) 
#one-vs-all SVM linear classifier 
lin_clf = svm.LinearSVC() 
#train the classifier using new hog features and training labels 
lin_clf.fit(hog_train_data, train_label) 
#likewise extract features for your testing data and test the classifier using the new features 
